In [1]:
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.InteractiveSession(config=config)

In [2]:
# Importing data
dn_input = np.load('/home/beto/Documents/projects/DoggoNet/DN-dataset.npz')

x_train = dn_input['X_train']
x_test = dn_input['X_test']
y_train = dn_input['y_train']
y_test = dn_input['y_test']

print('Training set shape:', x_train.shape)
print('Testing set shape:', x_test.shape)

# convert class
num_classes = 4
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Model initialization
model = keras.Sequential([keras.Input(shape=(225, 300, 3))])


# Defining building blocks
def add_conv_lyr(num_neurons, k_size, act_func, times=1):
    for i in range(times):
        model.add(layers.Conv2D(num_neurons, kernel_size=k_size, activation=act_func))


def add_max_pool_lyr(p_size, times=1):
    for i in range(times):
        model.add(layers.MaxPooling2D(pool_size=p_size))


def add_dense_lyr(num_neurons, act_func, times=1):
    for i in range(times):
        model.add(layers.Dense(num_neurons, activation=act_func))


# Adding logic to the model
add_conv_lyr(64, (3, 3), 'relu')
add_max_pool_lyr((2, 2))
add_conv_lyr(48, (3, 3), 'relu')
add_max_pool_lyr((2, 2))
add_conv_lyr(32, (3, 3), 'relu')
add_max_pool_lyr((2, 2))

add_dense_lyr(100, 'tanh', 3)

# Ending model
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
add_dense_lyr(num_classes, 'softmax')
model.summary()

batch_size = 34 * 3
epochs = 18

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
print('\n-------------------- MODEL COMPILED AND READY --------------------\n\n')

Training set shape: (3400, 225, 300, 3)
Testing set shape: (600, 225, 300, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 223, 298, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 149, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 147, 48)      27696     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 73, 48)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 71, 32)        13856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 35, 32)        0         
____________________________________________

In [3]:
import time
start_time = time.time()
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)
print((time.time() - start_time) / 60, 'minutes')

Epoch 1/18
30/30 [==============================] - 195s 6s/step - loss: 1.2390 - accuracy: 0.4556 - val_loss: 0.7936 - val_accuracy: 0.6618
Epoch 2/18
30/30 [==============================] - 196s 7s/step - loss: 0.6223 - accuracy: 0.7507 - val_loss: 0.5596 - val_accuracy: 0.7706
Epoch 3/18
30/30 [==============================] - 199s 7s/step - loss: 0.3726 - accuracy: 0.8647 - val_loss: 0.3367 - val_accuracy: 0.8853
Epoch 4/18
30/30 [==============================] - 197s 7s/step - loss: 0.2685 - accuracy: 0.9082 - val_loss: 0.2687 - val_accuracy: 0.9088
Epoch 5/18
30/30 [==============================] - 192s 6s/step - loss: 0.1540 - accuracy: 0.9497 - val_loss: 0.1867 - val_accuracy: 0.9235
Epoch 6/18
30/30 [==============================] - 193s 6s/step - loss: 0.0944 - accuracy: 0.9699 - val_loss: 0.1445 - val_accuracy: 0.9471
Epoch 7/18
30/30 [==============================] - 192s 6s/step - loss: 0.0622 - accuracy: 0.9817 - val_loss: 0.1603 - val_accuracy: 0.9471
Epoch 8/18
30

In [4]:
model.layers[0].weights

[<tf.Variable 'conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32, numpy=
 array([[[[-8.63986090e-02,  2.30772123e-02, -7.20379651e-02, ...,
            4.23043547e-03,  6.58809543e-02,  8.65409151e-02],
          [-1.15540558e-02,  3.34711298e-02,  5.04642017e-02, ...,
           -8.76927655e-03, -6.34524450e-02,  1.72126926e-02],
          [-2.30328180e-02, -4.20531817e-02, -9.47034545e-03, ...,
            5.35874106e-02,  8.79936218e-02, -7.04954416e-02]],
 
         [[ 6.12664595e-03, -5.12070246e-02, -6.05488382e-03, ...,
            4.86442558e-02,  3.47115844e-02,  5.20541072e-02],
          [ 2.41833609e-02,  8.87548830e-03, -6.84922375e-03, ...,
           -4.13448643e-03,  1.51918624e-02,  2.22651605e-02],
          [-2.42343135e-02, -7.12586418e-02,  8.05987492e-02, ...,
            7.90896825e-03, -5.59948832e-02,  2.94862986e-02]],
 
         [[ 9.47601050e-02, -1.42582646e-02, -8.39072466e-02, ...,
            6.29676580e-02, -4.93591540e-02,  5.34114428e-02],
          

In [5]:
#keras.backend.clear_session()